In [ ]:
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1805k      0 --:--:-- --:--:-- --:--:-- 1802k


In [ ]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [ ]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [ ]:


# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")



Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [ ]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [ ]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0281, 0.0224, 0.0136,  ..., 0.0346, 0.0447, 0.0223],
        [0.0025, 0.0376, 0.0024,  ..., 0.0321, 0.0203, 0.0120],
        [0.0359, 0.0456, 0.0129,  ..., 0.0419, 0.0420, 0.0133],
        ...,
        [0.0367, 0.0375, 0.0374,  ..., 0.0072, 0.0233, 0.0385],
        [0.0004, 0.0167, 0.0244,  ..., 0.0439, 0.0341, 0.0151],
        [0.0254, 0.0059, 0.0468,  ..., 0.0223, 0.0112, 0.0421]])
item_factors.weight tensor([[0.0415, 0.0100, 0.0095,  ..., 0.0470, 0.0245, 0.0083],
        [0.0354, 0.0447, 0.0255,  ..., 0.0041, 0.0484, 0.0077],
        [0.0144, 0.0420, 0.0099,  ..., 0.0066, 0.0262, 0.0276],
        ...,
        [0.0458, 0.0121, 0.0271,  ..., 0.0159, 0.0371, 0.0228],
        [0.0343, 0.0395, 0.0290,  ..., 0.0203, 0.0265, 0.0372],
        [0.0399, 0.0154, 0.0080,  ..., 0.0493, 0.0044, 0.0436]])


In [ ]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 0
iter #1 Loss: 0
iter #2 Loss: 0
iter #3 Loss: 0
iter #4 Loss: 0
iter #5 Loss: 0
iter #6 Loss: 0
iter #7 Loss: 0
iter #8 Loss: 0
iter #9 Loss: 0
iter #10 Loss: 0
iter #11 Loss: 0
iter #12 Loss: 0
iter #13 Loss: 0
iter #14 Loss: 0
iter #15 Loss: 0
iter #16 Loss: 0
iter #17 Loss: 0
iter #18 Loss: 0
iter #19 Loss: 0
iter #20 Loss: 0
iter #21 Loss: 0
iter #22 Loss: 0
iter #23 Loss: 0
iter #24 Loss: 0
iter #25 Loss: 0
iter #26 Loss: 0
iter #27 Loss: 0
iter #28 Loss: 0
iter #29 Loss: 0
iter #30 Loss: 0
iter #31 Loss: 0
iter #32 Loss: 0
iter #33 Loss: 0
iter #34 Loss: 0
iter #35 Loss: 0
iter #36 Loss: 0
iter #37 Loss: 0
iter #38 Loss: 0
iter #39 Loss: 0
iter #40 Loss: 0
iter #41 Loss: 0
iter #42 Loss: 0
iter #43 Loss: 0
iter #44 Loss: 0
iter #45 Loss: 0
iter #46 Loss: 0
iter #47 Loss: 0
iter #48 Loss: 0
iter #49 Loss: 0
iter #50 Loss: 0
iter #51 Loss: 0
iter #52 Loss: 0
iter #53 Loss: 0
iter #54 Loss: 0
iter #55 Loss: 0
iter #56 Loss: 0
iter #57 Loss: 0
iter #58 Loss: 0
iter #5

In [ ]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0281, 0.0224, 0.0136,  ..., 0.0346, 0.0447, 0.0223],
        [0.0025, 0.0376, 0.0024,  ..., 0.0321, 0.0203, 0.0120],
        [0.0359, 0.0456, 0.0129,  ..., 0.0419, 0.0420, 0.0133],
        ...,
        [0.0367, 0.0375, 0.0374,  ..., 0.0072, 0.0233, 0.0385],
        [0.0004, 0.0167, 0.0244,  ..., 0.0439, 0.0341, 0.0151],
        [0.0254, 0.0059, 0.0468,  ..., 0.0223, 0.0112, 0.0421]])
item_factors.weight tensor([[0.0415, 0.0100, 0.0095,  ..., 0.0470, 0.0245, 0.0083],
        [0.0354, 0.0447, 0.0255,  ..., 0.0041, 0.0484, 0.0077],
        [0.0144, 0.0420, 0.0099,  ..., 0.0066, 0.0262, 0.0276],
        ...,
        [0.0458, 0.0121, 0.0271,  ..., 0.0159, 0.0371, 0.0228],
        [0.0343, 0.0395, 0.0290,  ..., 0.0203, 0.0265, 0.0372],
        [0.0399, 0.0154, 0.0080,  ..., 0.0493, 0.0044, 0.0436]])


In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [ ]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [ ]:


'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])



Cluster #0
	 Terminator 2: Judgment Day (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Back to the Future (1985)
	 American History X (1998)
	 Aliens (1986)
	 Austin Powers: The Spy Who Shagged Me (1999)
	 Ghost (1990)
	 Bourne Identity, The (2002)
	 Star Wars: Episode II - Attack of the Clones (2002)
	 Mummy, The (1999)
Cluster #1
	 Star Wars: Episode IV - A New Hope (1977)
	 Godfather, The (1972)
	 Lord of the Rings: The Return of the King, The (2003)
	 Aladdin (1992)
	 Finding Nemo (2003)
	 Indiana Jones and the Last Crusade (1989)
	 Titanic (1997)
	 Blade Runner (1982)
	 Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
	 Willy Wonka & the Chocolate Factory (1971)
Cluster #2
	 Forrest Gump (1994)
	 Matrix, The (1999)
	 Shrek (2001)
	 Dances with Wolves (1990)
	 Die Hard (1988)
	 Pretty Woman (1990)
	 GoldenEye (1995)
	 Goodfellas (1990)
	 Truman Show, The (1998)
	 Jumanji (1995)
Cluster #3
	 Fight Club (1999)
	 Mission: Impossible (1996)
	 Die Hard: With a Vengeance (1995)
	 Inception (